В домашнем задании нужно решить задачу классификации наличия болезни сердца у пациентов наиболее эффективно. Данные для обучения моделей необходимо загрузить самостоятельно с сайта. Целевая переменная – наличие болезни сердца (HeartDisease). Она принимает значения 0 или 1 в зависимости от отсутствия или наличия болезни соответственно. Подробное описание признаков можно прочесть в описании датасета на сайте. Для выполнения работы не обязательно вникать в медицинские показатели.

Получите данные и загрузите их в рабочую среду. (Jupyter Notebook или другую)
Подготовьте датасет к обучению моделей:
a) Категориальные переменные переведите в цифровые значения. Можно использовать pd.get_dummies, preprocessing.LabelEncoder. Старайтесь не использовать для этой задачи циклы.

Разделите выборку на обучающее и тестовое подмножество. 80% данных оставить на обучающее множество, 20% на тестовое.
Обучите модель логистической регрессии с параметрами по умолчанию.
Подсчитайте основные метрики модели. Используйте следующие метрики и функцию:
cross_validate(…, cv=10, scoring=[‘accuracy’,‘recall’,‘precision’,‘f1’])
Оптимизируйте 3-4 параметра модели:
a) Используйте GridSearchCV.
b) Используйте RandomizedSearchCV.
c) *Добавьте в п. 6b 2-5 моделей классификации и вариации их параметров.
d) Повторите п. 5 после каждого итогового изменения параметров.

In [96]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split 
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

In [5]:
df = pd.read_csv('heart.csv')

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 918 entries, 0 to 917
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Age             918 non-null    int64  
 1   Sex             918 non-null    object 
 2   ChestPainType   918 non-null    object 
 3   RestingBP       918 non-null    int64  
 4   Cholesterol     918 non-null    int64  
 5   FastingBS       918 non-null    int64  
 6   RestingECG      918 non-null    object 
 7   MaxHR           918 non-null    int64  
 8   ExerciseAngina  918 non-null    object 
 9   Oldpeak         918 non-null    float64
 10  ST_Slope        918 non-null    object 
 11  HeartDisease    918 non-null    int64  
dtypes: float64(1), int64(6), object(5)
memory usage: 86.2+ KB


In [7]:
# какое кол-во уникальных значений принимают категориальные столбцы
for c in df.select_dtypes(exclude=['int64']).columns:
    print(f'{c}: {len(df[c].unique())}')

Sex: 2
ChestPainType: 4
RestingECG: 3
ExerciseAngina: 2
Oldpeak: 53
ST_Slope: 3


In [8]:
df_ = pd.get_dummies(df, columns=['Sex', 'ChestPainType', 'RestingECG', 'ExerciseAngina', 'ST_Slope'])
df_.drop(columns = ['Sex_F', 'ExerciseAngina_N'], inplace = True)

df_.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 918 entries, 0 to 917
Data columns (total 19 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Age                918 non-null    int64  
 1   RestingBP          918 non-null    int64  
 2   Cholesterol        918 non-null    int64  
 3   FastingBS          918 non-null    int64  
 4   MaxHR              918 non-null    int64  
 5   Oldpeak            918 non-null    float64
 6   HeartDisease       918 non-null    int64  
 7   Sex_M              918 non-null    uint8  
 8   ChestPainType_ASY  918 non-null    uint8  
 9   ChestPainType_ATA  918 non-null    uint8  
 10  ChestPainType_NAP  918 non-null    uint8  
 11  ChestPainType_TA   918 non-null    uint8  
 12  RestingECG_LVH     918 non-null    uint8  
 13  RestingECG_Normal  918 non-null    uint8  
 14  RestingECG_ST      918 non-null    uint8  
 15  ExerciseAngina_Y   918 non-null    uint8  
 16  ST_Slope_Down      918 non

Разделите выборку на обучающее и тестовое подмножество. 80% данных оставить на обучающее множество, 20% на тестовое.
Обучите модель логистической регрессии с параметрами по умолчанию.

In [9]:
model = LogisticRegression(solver = 'liblinear')

X = df_.drop(columns = ['HeartDisease'])
y = df_['HeartDisease']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

model.fit(X_train, y_train)


LogisticRegression(solver='liblinear')

In [10]:
model.score(X_train, y_train) 

0.8582554517133957

Подсчитайте основные метрики модели. Используйте следующие метрики и функцию:
cross_validate(…, cv=10, scoring=[‘accuracy’,‘recall’,‘precision’,‘f1’]

In [11]:
cv_res = cross_validate(model, X, y, cv=10, scoring=['accuracy','recall','precision','f1'])

In [12]:
cv_res

{'fit_time': array([0.00897503, 0.00598454, 0.00597334, 0.00698137, 0.00598288,
        0.00498605, 0.00598836, 0.00639224, 0.00598359, 0.00498629]),
 'score_time': array([0.00900936, 0.00695705, 0.00598383, 0.00498724, 0.00598359,
        0.00701141, 0.00498867, 0.00498724, 0.004987  , 0.0070107 ]),
 'test_accuracy': array([0.83695652, 0.92391304, 0.88043478, 0.95652174, 0.86956522,
        0.84782609, 0.88043478, 0.83695652, 0.73626374, 0.76923077]),
 'test_recall': array([0.78431373, 0.90196078, 0.8627451 , 0.96078431, 0.98039216,
        1.        , 0.98039216, 0.82352941, 0.76      , 0.7       ]),
 'test_precision': array([0.90909091, 0.95833333, 0.91666667, 0.96078431, 0.81967213,
        0.78461538, 0.83333333, 0.875     , 0.76      , 0.85365854]),
 'test_f1': array([0.84210526, 0.92929293, 0.88888889, 0.96078431, 0.89285714,
        0.87931034, 0.9009009 , 0.84848485, 0.76      , 0.76923077])}

Оптимизируйте 3-4 параметра модели:
a) Используйте GridSearchCV.
b) Используйте RandomizedSearchCV.
c) *Добавьте в п. 6b 2-5 моделей классификации и вариации их параметров.
d) Повторите п. 5 после каждого итогового изменения параметров.

In [89]:

parameters = {'C':np.logspace(-3,3,20), 'penalty':['l2'], 'solver': ['liblinear'], 'intercept_scaling': np.logspace(-3,3,20) } 
grid = GridSearchCV(model, parameters, cv=10, scoring='accuracy')
grid.fit(X, y)



GridSearchCV(cv=10, estimator=LogisticRegression(solver='liblinear'),
             param_grid={'C': array([1.00000000e-03, 2.06913808e-03, 4.28133240e-03, 8.85866790e-03,
       1.83298071e-02, 3.79269019e-02, 7.84759970e-02, 1.62377674e-01,
       3.35981829e-01, 6.95192796e-01, 1.43844989e+00, 2.97635144e+00,
       6.15848211e+00, 1.27427499e+01, 2.63665090e+01, 5.45559478e+01,
       1.12883789e+02, 2.33...
                         'intercept_scaling': array([1.00000000e-03, 2.06913808e-03, 4.28133240e-03, 8.85866790e-03,
       1.83298071e-02, 3.79269019e-02, 7.84759970e-02, 1.62377674e-01,
       3.35981829e-01, 6.95192796e-01, 1.43844989e+00, 2.97635144e+00,
       6.15848211e+00, 1.27427499e+01, 2.63665090e+01, 5.45559478e+01,
       1.12883789e+02, 2.33572147e+02, 4.83293024e+02, 1.00000000e+03]),
                         'penalty': ['l2'], 'solver': ['liblinear']},
             scoring='accuracy')

In [90]:
grid.best_estimator_

LogisticRegression(C=0.6951927961775606, intercept_scaling=1.438449888287663,
                   solver='liblinear')

In [94]:
model = LogisticRegression(C=0.6951927961775606, intercept_scaling=1.438449888287663, solver='liblinear')
model.fit(X_train, y_train)

model.score(X_train, y_train) 

0.8598130841121495

In [98]:
y_pred =model.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.83      0.90      0.86       112
           1       0.93      0.87      0.90       164

    accuracy                           0.88       276
   macro avg       0.88      0.89      0.88       276
weighted avg       0.89      0.88      0.88       276



In [54]:
parameters = {'C':np.logspace(-3,3,20), 'penalty':['l2'], 'solver': ['liblinear'], 'intercept_scaling': np.logspace(-3,3,20) } 
grid = RandomizedSearchCV(model, parameters, cv=10, scoring='accuracy')
grid.fit(X, y)

RandomizedSearchCV(cv=10, estimator=LogisticRegression(solver='liblinear'),
                   param_distributions={'C': array([1.00000000e-03, 2.06913808e-03, 4.28133240e-03, 8.85866790e-03,
       1.83298071e-02, 3.79269019e-02, 7.84759970e-02, 1.62377674e-01,
       3.35981829e-01, 6.95192796e-01, 1.43844989e+00, 2.97635144e+00,
       6.15848211e+00, 1.27427499e+01, 2.63665090e+01, 5.45559478e+01,
       1.12...
                                        'intercept_scaling': array([1.00000000e-03, 2.06913808e-03, 4.28133240e-03, 8.85866790e-03,
       1.83298071e-02, 3.79269019e-02, 7.84759970e-02, 1.62377674e-01,
       3.35981829e-01, 6.95192796e-01, 1.43844989e+00, 2.97635144e+00,
       6.15848211e+00, 1.27427499e+01, 2.63665090e+01, 5.45559478e+01,
       1.12883789e+02, 2.33572147e+02, 4.83293024e+02, 1.00000000e+03]),
                                        'penalty': ['l2'],
                                        'solver': ['liblinear']},
                   scoring='accuracy')

In [55]:
grid.cv_results_['mean_test_score']

array([0.76029623, 0.84947444, 0.79513856, 0.8505614 , 0.85273531,
       0.8505614 , 0.81691352, 0.7308409 , 0.85162446, 0.764656  ])

In [30]:
grid.best_estimator_

LogisticRegression(C=12.742749857031322, intercept_scaling=0.3359818286283781,
                   solver='liblinear')

In [99]:
model = LogisticRegression(C=12.742749857031322, intercept_scaling=0.3359818286283781, solver='liblinear')
model.fit(X_train, y_train)

model.score(X_train, y_train) 

0.8613707165109035

In [100]:
y_pred =model.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.83      0.88      0.86       112
           1       0.92      0.88      0.90       164

    accuracy                           0.88       276
   macro avg       0.87      0.88      0.88       276
weighted avg       0.88      0.88      0.88       276



In [77]:
knn = KNeighborsClassifier()
parameters = dict(n_neighbors=list(range(1, 30)), p=np.logspace(-3,3,num=20), metric=['minkowski'])
grid = RandomizedSearchCV(knn, parameters, cv=10, n_iter=10, scoring='accuracy')
grid.fit(X, y)

RandomizedSearchCV(cv=10, estimator=KNeighborsClassifier(),
                   param_distributions={'metric': ['minkowski'],
                                        'n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8,
                                                        9, 10, 11, 12, 13, 14,
                                                        15, 16, 17, 18, 19, 20,
                                                        21, 22, 23, 24, 25, 26,
                                                        27, 28, 29],
                                        'p': array([1.00000000e-03, 2.06913808e-03, 4.28133240e-03, 8.85866790e-03,
       1.83298071e-02, 3.79269019e-02, 7.84759970e-02, 1.62377674e-01,
       3.35981829e-01, 6.95192796e-01, 1.43844989e+00, 2.97635144e+00,
       6.15848211e+00, 1.27427499e+01, 2.63665090e+01, 5.45559478e+01,
       1.12883789e+02, 2.33572147e+02, 4.83293024e+02, 1.00000000e+03])},
                   scoring='accuracy')

In [78]:
grid.best_estimator_

KNeighborsClassifier(n_neighbors=15, p=12.742749857031322)

In [79]:
grid.cv_results_['mean_test_score']

array([0.70351171, 0.70246058, 0.66433349,        nan,        nan,
       0.46186097, 0.69265409,        nan,        nan, 0.7024128 ])

In [101]:
model = KNeighborsClassifier(n_neighbors=15, p=12.742749857031322)
model.fit(X_train, y_train)

model.score(X_train, y_train) 

0.7305295950155763

In [102]:
y_pred =model.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.62      0.71      0.66       112
           1       0.78      0.70      0.74       164

    accuracy                           0.70       276
   macro avg       0.70      0.70      0.70       276
weighted avg       0.71      0.70      0.71       276



In [84]:
rfc = RandomForestClassifier(max_depth=2, random_state=42)

parameters = {'n_estimators': [200, 700], 'max_features': ['auto', 'sqrt', 'log2'], 'ccp_alpha':np.logspace(-3,3,20)}
grid = RandomizedSearchCV(rfc, parameters, cv= 10,scoring='accuracy')
grid.fit(X, y)


RandomizedSearchCV(cv=10,
                   estimator=RandomForestClassifier(max_depth=2,
                                                    random_state=42),
                   param_distributions={'ccp_alpha': array([1.00000000e-03, 2.06913808e-03, 4.28133240e-03, 8.85866790e-03,
       1.83298071e-02, 3.79269019e-02, 7.84759970e-02, 1.62377674e-01,
       3.35981829e-01, 6.95192796e-01, 1.43844989e+00, 2.97635144e+00,
       6.15848211e+00, 1.27427499e+01, 2.63665090e+01, 5.45559478e+01,
       1.12883789e+02, 2.33572147e+02, 4.83293024e+02, 1.00000000e+03]),
                                        'max_features': ['auto', 'sqrt',
                                                         'log2'],
                                        'n_estimators': [200, 700]},
                   scoring='accuracy')

In [85]:
grid.best_estimator_

RandomForestClassifier(ccp_alpha=0.001, max_depth=2, n_estimators=200,
                       random_state=42)

In [86]:
grid.cv_results_['mean_test_score']

array([0.55336837, 0.55336837, 0.55336837, 0.55336837, 0.81347348,
       0.55336837, 0.81889632, 0.55336837, 0.83852126, 0.55336837])

In [103]:
model = RandomForestClassifier(ccp_alpha=0.001, max_depth=2, n_estimators=200, random_state=42)
model.fit(X_train, y_train)

model.score(X_train, y_train) 

0.8364485981308412

In [104]:
y_pred =model.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.79      0.80      0.80       112
           1       0.86      0.85      0.86       164

    accuracy                           0.83       276
   macro avg       0.83      0.83      0.83       276
weighted avg       0.83      0.83      0.83       276



В ходе работы применили на практике алгоритмы поиска оптимальной параметрии, получили улучшение на модели LogisticRegression. на остальных моделях результы похуже чем в работе "Домашнее задание по теме «Ансамблирование»"